In [7]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from qiskit.circuit import ParameterVector

In [8]:
df = datasets.load_iris()
dff = datasets.load_iris()
dff = pd.DataFrame(data=np.c_[dff['data'], dff['target']], columns=dff['feature_names'] + ['target'])
df = pd.DataFrame(data=np.c_[df['data'], df['target']], columns=df['feature_names'] + ['target'])
df_columns = df.columns.values[:-1] 
df.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333,1.000000
std,0.828066,0.435866,1.765298,0.762238,0.819232
min,4.300000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000
50%,5.800000,3.000000,4.350000,1.300000,1.000000
75%,6.400000,3.300000,5.100000,1.800000,2.000000
max,7.900000,4.400000,6.900000,2.500000,2.000000


In [9]:
x_train, x_test, y_train, y_test = train_test_split(df[df_columns], df['target'], test_size=0.30, random_state=0, stratify=df['target'])
x_val, x_test, y_val, y_test = train_test_split(x_train, y_train, test_size=0.5, random_state=0, stratify=y_train)

y_train.value_counts(), y_val.value_counts(), y_test.value_counts()

(target
 2.0    35
 0.0    35
 1.0    35
 Name: count, dtype: int64,
 target
 2.0    18
 1.0    17
 0.0    17
 Name: count, dtype: int64,
 target
 0.0    18
 1.0    18
 2.0    17
 Name: count, dtype: int64)

In [13]:
dff[dff.duplicated(keep=False)]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
101,5.8,2.7,5.1,1.9,2.0
142,5.8,2.7,5.1,1.9,2.0


In [5]:
dff.drop_duplicates(inplace=True)

# Preprocessing

In this project we decided that we will use ``angle encoding`` because we don't have that many quibits to work with. Since angle enncoding needs the data to be on a scale between $[0, 2\pi]$, we will scale the data accordingly

In [ ]:
# custom transformers for the pipeline
class RemoveDuplicates(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        df = pd.DataFrame(X)
        df = df.drop_duplicates()
        return df.values

In [8]:
scaler = MinMaxScaler(feature_range=(0, 2*np.pi))
remove_duplicates = RemoveDuplicates()

pipe = Pipeline([
    ('scaler', scaler),
    ('remove_duplicates', remove_duplicates)
])

In [9]:
x_train = pipe.fit_transform(x_train)
x_val = pipe.transform(x_val)
x_test = pipe.transform(x_test)

# choosing a loss function

In [ ]:
class IrisQNN:
    def __init__(self, n_qubits, n_layers):
        self.n_qubits = n_qubits
        self.n_layers = n_layers
        self.n_params = n_qubits * n_layers
        self.params = ParameterVector('θ', self.n_params)
    
    def angle_encoding(self, qc, input_features):
        number_of_qubits = qc.num_qubits
        for qubit in range(number_of_qubits):
            qc.rx(input_features[qubit], qubit)
        qc.barrier()
        return qc

    def add_variational_layer_real_amplitude(self, qc, layer_idx):
        param_offset = layer_idx * self.n_qubits
        for i in range(self.n_qubits):
            qc.ry(self.params[param_offset + i], i)
        
        qc.barrier()
        
        for i in range(self.n_qubits - 1):
            qc.cx(i, i+1)
            
        if layer_idx < self.n_layers - 1:
            qc.barrier()
            
        return qc

    def create_circuit(self, input_features, variational_params=None):
        qc = QuantumCircuit(self.n_qubits)
        
        self.angle_encoding(qc, input_features)
        
        for layer in range(self.n_layers):
            self.add_variational_layer_real_amplitude(qc, layer)
        
        if variational_params is not None:
            qc.bind_parameters({self.params: variational_params})

        return qc
    
    def _get_parameter_count(self):
        return self.n_params

In [4]:
qnn = IrisQNN(n_qubits=4, n_layers=2)
feature = x_train[0]
qc = qnn.create_circuit(feature)
qc.draw('mpl')

NameError: name 'IrisQNN' is not defined

In [ ]:
def split_into_batches(data, batch_size):
    for i in range(0, len(data), batch_size):
        yield data[i:i+batch_size]


[[1, 2, 3], [4, 5, 6], [7, 8, 9], [10]]

In [ ]:
epochs = 1
x_train_sub = x_train[:1]

for epoch in range(epochs):
    




array([0.41719255, 0.69787902, 0.96756243, 0.18044916, 0.3299218 ,
       0.18911844, 0.70694054, 0.13754617])

In [ ]:
#TODO Implent funciton for initialising the weights
#TODO Implement funciton for creating circuits for all the training data
#TODO implement function for calculating the cost function